# Wavelength Loss dependence using BraggExact

Here for the N2 configuration from Bird.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

ts = [15*2.7183333333333333e-6, 10e-6,
      15*2.7183333333333333e-6, 10e-6, 15*2.7183333333333333e-6]

mats = ['air', 'glass', 'air', 'glass', 'air']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x:n_glass, lambda x:n_air]
maxhs = [100,100,100,100,100]




# Four Layer Study: Glass outside Air (Outside Cladding)

In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, ns=ns, mats=mats, maxhs=maxhs, wl=wl)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low) +0*1j
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess = 1/.999999 * guess.real + imag * 1j
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag  # set current imaginary part as guess for next one
                flag=False
        except RuntimeError:
            guess = .999999 * guess.real + 1j * imag
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 10e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(8,5))
ax = plt.gca()

ax.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
ym,yM = ax.get_ylim()
for l in ls:
    ax.plot([l,l], [0, yM],  linewidth=1, linestyle=':')

In [ ]:
np.save('/home/pv/local/convergence/bragg_fiber/wavelength/N3/exact_scaled_betas', betas1)